In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

## Setting the LLM

In [ ]:
with open("openai_api.txt", "r") as f:
    OPENAI_API = f.read()

llm = OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    openai_api_key = OPENAI_API
)

## Combining Chains

The next step after calling a language model is to make a series of calls to a language model. We can do this using `sequential chains`, which are chains that execute their links in a predefined order.

Sequential chains allow you to connect multiple chains and compose them into pipelines that execute some specific scenario. There are two types of sequential chains:
1. `SimpleSequentialChain`: The simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.
2. `SequentialChain`: A more general form of sequential chains, allowing for multiple inputs/outputs

In [ ]:
## Creating the First Chain

prompt = PromptTemplate(
    input_variables=["company"],
    template="What is a good name for {company}?",
)

chain = LLMChain(
    llm = llm,
    prompt = prompt
)

In [ ]:
## Creating the Second Chain

second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchphrase for the following company: {company_name}",
)

chain_two = LLMChain(
    llm = llm,
    prompt = second_prompt
)

In [ ]:
## Combining those two Chains

overall_chain = SimpleSequentialChain(
    chains = [chain, chain_two],
    verbose = True
)

overall_chain.run("colorful socks")

In [ ]:
## Creating More Complex Sequential Chains

# Chain .1
synopsis_template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
synopsis_prompt_template = PromptTemplate(
    template = synopsis_template,
    input_variables = ["title", "era"]
)
synopsis_chain = LLMChain(
    llm = llm,
    prompt = synopsis_prompt_template,
    output_key = "synopsis"
)


# Chain .2
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(
    template = template,
    input_variables = ["synopsis"]
)
review_chain = LLMChain(
    llm = llm,
    prompt = prompt_template,
    output_key = "review"
)

# Overall Chain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})